In [ ]:
!pip install -q SQLAlchemy openpyxl


import pandas as pd
import numpy as np
import re
from datetime import datetime
import os

try:
    from sqlalchemy import create_engine
    HAVE_SQLALCHEMY = True
except:
    HAVE_SQLALCHEMY = False


In [ ]:
from google.colab import files
uploaded = files.upload()
RAW_PATH = next(iter(uploaded.keys()))
print("📄 Raw File Loaded:", RAW_PATH)


Saving Citi_Case_Management_Raw_WithDate.xlsx to Citi_Case_Management_Raw_WithDate.xlsx
📄 Raw File Loaded: Citi_Case_Management_Raw_WithDate.xlsx


In [ ]:
OUT_DIR = "/content/clean_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

DB_PATH = "/content/clean_outputs/citi_case_clean.db"

print("📂 Output Folder:", OUT_DIR)

📂 Output Folder: /content/clean_outputs


In [ ]:
raw = pd.read_excel(RAW_PATH, sheet_name=None)
print("📑 Raw Sheets Found:", list(raw.keys()))

📑 Raw Sheets Found: ['Customer_Details', 'CreditCard_Account_Details', 'Loan_Application_Cases', 'Customer_Service_Requests', 'Escalations', 'Payments_Transactions', 'Case_Audit_Log']


In [ ]:

cust  = raw["Customer_Details"].copy()
acct  = raw["CreditCard_Account_Details"].copy()
cases = raw["Loan_Application_Cases"].copy()
reqs  = raw["Customer_Service_Requests"].copy()
escal = raw["Escalations"].copy()
txns  = raw["Payments_Transactions"].copy()
aud   = raw["Case_Audit_Log"].copy()


In [ ]:
# Standard list of known date formats
DATE_FORMATS = [
    "%Y-%m-%d", "%d/%m/%Y", "%m-%d-%Y", "%Y/%m/%d",
    "%d-%m-%Y", "%d-%b-%y", "%d-%b-%Y"
]

def to_date(series):
    """Convert mixed-format date values into proper datetime type."""
    def parse(x):
        if pd.isna(x): return pd.NaT
        s = str(x).strip()

        s = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', s)

        for fmt in DATE_FORMATS:
            try:
                return datetime.strptime(s, fmt)
            except:
                pass

        return pd.to_datetime(s, errors='coerce')
    return series.apply(parse)

def clean_currency_to_float(s):
    """Convert messy currency strings like 'AED 1,200' and '20 K' → float."""
    if pd.isna(s): return np.nan
    s = str(s).upper().replace("AED","").replace(",","").strip()
    if "K" in s:
        return float(s.replace("K","")) * 1000
    try:
        return float(s)
    except:
        return np.nan

def dedupe(df, key):
    """Remove duplicate rows → keep first appearance only."""
    return df.drop_duplicates(subset=[key], keep="first")


In [ ]:

cust["RecordDate"] = to_date(cust["RecordDate"])
cust["CustomerID"] = cust["CustomerID"].astype(str).str.strip()
cust["City"] = cust["City"].astype(str).str.title()
cust["DateOfBirth"] = to_date(cust["DateOfBirth"])
cust["CustomerSince"] = to_date(cust["CustomerSince"])

cust["RiskScore_num"] = pd.to_numeric(cust["RiskScore"], errors="coerce")

cust_clean = dedupe(cust, "CustomerID")

print("Customer rows after cleaning:", len(cust_clean))


Customer rows after cleaning: 100


In [ ]:

# clean accounts table

def clean_currency_to_float(series):
    """Convert messy currency strings into numeric AED values."""
    def parse(s):
        if pd.isna(s):
            return np.nan
        s = str(s).upper().replace("AED", "").replace(",", "").strip()


        if "K" in s:
            s = s.replace("K", "").strip()
            try:
                return float(s) * 1000
            except:
                return np.nan


        try:
            return float(s)
        except:
            return np.nan

    return series.apply(parse)



acct["RecordDate"] = to_date(acct["RecordDate"])


acct["AccountID"] = acct["AccountID"].astype(str).str.strip()
acct["CustomerID"] = acct["CustomerID"].astype(str).str.strip()


acct["OpenDate"] = to_date(acct["OpenDate"])
acct["LastStatementDate"] = to_date(acct["LastStatementDate"])


card_map = {
    "PLATINUM": "Platinum",
    "GOLD": "Gold",
    "TITANIUM": "Titanium",
    "PLATNUM": "Platinum"
}
acct["CardType_std"] = acct["CardType"].astype(str).str.upper().map(card_map).fillna(acct["CardType"])


status_map = {"ACTIVE": "Active", "CLOSED": "Closed"}
acct["Status_std"] = acct["Status"].astype(str).str.upper().map(status_map).fillna("Active")


acct["CreditLimit_num"] = clean_currency_to_float(acct["CreditLimit"])
acct["Balance_num"] = clean_currency_to_float(acct["Balance"])


acct_clean = acct.drop_duplicates(subset=["AccountID"], keep="first")

print("✅ Accounts cleaned successfully!")
print("🔢 Rows after cleaning:", len(acct_clean))


acct_clean.head(5)


✅ Accounts cleaned successfully!
🔢 Rows after cleaning: 120


,RecordDate,AccountID,CustomerID,CardType,CardNumber,CreditLimit,Balance,Status,OpenDate,LastStatementDate,CardType_std,Status_std,CreditLimit_num,Balance_num
0,2025-09-27,CITI-ACC-0001,CITI-CUST-0080,GOLD,5241-XXXX-XXXX-2178,20 K,1500,Active,2020-05-01,2025-10-01,Gold,Active,20000.0,1500.0
1,2025-06-24,CITI-ACC-0002,CITI-CUST-0059,platnum,5241-XXXX-XXXX-2841,NaN,"AED 1,500",active,2020-01-05,2025-10-01,Platinum,Active,NaN,1500.0
2,2025-09-08,CITI-ACC-0003,CITI-CUST-0023,Gold,5241-XXXX-XXXX-9815,20000,NaN,Active,2020-05-01,2025-10-01,Gold,Active,20000.0,NaN
3,2025-08-02,CITI-ACC-0004,CITI-CUST-0099,Platinum,5241-XXXX-XXXX-6921,20 K,"AED 1,500",active,2020-01-05,2025-10-01,Platinum,Active,20000.0,1500.0
4,2025-06-28,CITI-ACC-0005,CITI-CUST-0095,platnum,5241-XXXX-XXXX-4610,"AED 20,000","AED 1,500",active,NaT,2025-10-01,Platinum,Active,20000.0,1500.0


In [ ]:
# clean customer table
cust["RecordDate"] = to_date(cust["RecordDate"])
cust["DateOfBirth"] = to_date(cust["DateOfBirth"])
cust["CustomerSince"] = to_date(cust["CustomerSince"])

cust["CustomerID"] = cust["CustomerID"].astype(str).str.strip()
cust["FullName"] = cust["FullName"].astype(str).str.title().str.strip()
cust["City"] = cust["City"].astype(str).str.title().str.strip()


cust["RiskScore_num"] = pd.to_numeric(cust["RiskScore"], errors="coerce")

cust_clean = dedupe(cust, "CustomerID")

print("✅ Clean Customers:", len(cust_clean))
cust_clean.head(3)


✅ Clean Customers: 100


,RecordDate,CustomerID,FullName,DateOfBirth,Email,Phone,Address,City,Country,CustomerSince,RiskScore,RiskScore_num
0,2025-09-21,CITI-CUST-0001,Ravi Patel,1989-04-12,user0@citibank.ae,9.715069e+11,Khalifa Street,Sharjah,UAE,2018-06-15,784.0,784.0
1,2025-10-13,CITI-CUST-0002,Omar Hassan,NaT,user1@citibank.ae,9.715016e+11,Marina View,Abu Dhabi,UAE,NaT,749.0,749.0
2,2025-08-10,CITI-CUST-0003,Priya Nair,1989-04-12,test@example.com,NaN,Business Bay,Abu Dhabi,UAE,NaT,719.0,719.0


In [ ]:
# clean loan application cases table
cases["RecordDate"] = to_date(cases["RecordDate"])
cases["ApplicationDate"] = to_date(cases["ApplicationDate"])
cases["DecisionDate"] = to_date(cases["DecisionDate"])

cases["CaseID"] = cases["CaseID"].astype(str).str.strip()
cases["CustomerID"] = cases["CustomerID"].astype(str).str.strip()
cases["AccountID"] = cases["AccountID"].astype(str).str.strip()


status_map_case = {"UNDER REVIEW":"Under Review","APPROVED":"Approved","REJECTED":"Rejected"}
cases["CaseStatus_std"] = cases["CaseStatus"].astype(str).str.upper().map(status_map_case)


cases["LoanAmount_num"] = clean_currency_to_float(cases["LoanAmount"])

cases_clean = dedupe(cases, "CaseID")

print("✅ Clean Cases:", len(cases_clean))
cases_clean.head(3)


✅ Clean Cases: 150


,RecordDate,CaseID,CustomerID,AccountID,ApplicationDate,LoanAmount,Purpose,CaseStatus,AssignedTo,DecisionDate,Decision,CaseStatus_std,LoanAmount_num
0,2025-07-22,CITI-CASE-0001,CITI-CUST-0008,CITI-ACC-0055,NaT,"AED 15,000",Travel,UnderReview,CreditOfficer01,2025-09-12,NaN,NaN,15000.0
1,2025-08-28,CITI-CASE-0002,CITI-CUST-0067,CITI-ACC-0104,2025-09-10,NaN,Medical,Under Review,CreditOfficer01,2025-09-12,Rejected,Under Review,NaN
2,2025-10-07,CITI-CASE-0003,CITI-CUST-0008,CITI-ACC-0115,2025-10-09,"AED 15,000",Medical,Under Review,CreditOfficer01,2025-09-12,Rejected,Under Review,15000.0


In [ ]:
# clean customer service requests table
reqs["RecordDate"] = to_date(reqs["RecordDate"])
reqs["RequestDate"] = to_date(reqs["RequestDate"])
reqs["ResolutionDate"] = to_date(reqs["ResolutionDate"])

reqs["RequestID"] = reqs["RequestID"].astype(str).str.strip()
reqs["CustomerID"] = reqs["CustomerID"].astype(str).str.strip()
reqs["AccountID"] = reqs["AccountID"].astype(str).str.strip()

# ✅ Standardize request categories
reqs["RequestType_std"] = reqs["RequestType"].astype(str).str.title()

# ✅ Standardize priority labels
reqs["Priority_std"] = reqs["Priority"].astype(str).str.title()

reqs_clean = dedupe(reqs, "RequestID")

print("✅ Clean Requests:", len(reqs_clean))
reqs_clean.head(3)


✅ Clean Requests: 200


,RecordDate,RequestID,CustomerID,AccountID,RequestType,RequestDate,Priority,Status,AssignedTo,ResolutionDate,Comments,RequestType_std,Priority_std
0,2025-07-27,CITI-SR-0001,CITI-CUST-0083,CITI-ACC-0016,NaN,NaT,low,Resolved,CSR_Alice,2025-03-10,NaN,Nan,Low
1,2025-09-13,CITI-SR-0002,CITI-CUST-0003,CITI-ACC-0045,Card Replacement,NaT,low,NaN,CSR_Ben,2025-10-03,NaN,Card Replacement,Low
2,2025-05-13,CITI-SR-0003,CITI-CUST-0071,CITI-ACC-0072,Card Replacement,2025-10-01,Hgh,Closed,CSR_Alice,2025-10-03,Customer contacted,Card Replacement,Hgh


In [ ]:
# clean escalations table
escal["RecordDate"] = to_date(escal["RecordDate"])
escal["RaisedDate"] = to_date(escal["RaisedDate"])
escal["ResolutionDate"] = to_date(escal["ResolutionDate"])

escal["EscalationID"] = escal["EscalationID"].astype(str).str.strip()

escal_clean = dedupe(escal, "EscalationID")

print("✅ Clean Escalations:", len(escal_clean))
escal_clean.head(3)


✅ Clean Escalations: 50


,RecordDate,EscalationID,RelatedCaseID,RequestID,RaisedBy,RaisedDate,EscalationReason,EscalationLevel,Status,ResolvedBy,ResolutionDate
0,2025-08-16,CITI-ESC-0001,CITI-CASE-0007,CITI-SR-0099,NaN,NaT,Customer card not received,Level 1,Open,Manager_Susan,2025-10-07
1,2025-10-21,CITI-ESC-0002,CITI-CASE-0106,CITI-SR-0091,CSR_Ben,NaT,Customer card not received,NaN,Open,Manager_Tom,2025-10-07
2,2025-05-10,CITI-ESC-0003,NaN,CITI-SR-0012,NaN,NaT,NaN,NaN,NaN,Manager_Susan,NaT


In [ ]:
# clean payments transactions table
txns["RecordDate"] = to_date(txns["RecordDate"])
txns["TransactionDate"] = to_date(txns["TransactionDate"])

txns["TransactionID"] = txns["TransactionID"].astype(str).str.strip()
txns["AccountID"] = txns["AccountID"].astype(str).str.strip()


txns["Type_std"] = txns["Type"].astype(str).str.title()
txns["Channel_std"] = txns["Channel"].astype(str).str.title()


txns["Amount_num"] = clean_currency_to_float(txns["Amount"])

txns_clean = dedupe(txns, "TransactionID")

print("✅ Clean Transactions:", len(txns_clean))
txns_clean.head(3)


✅ Clean Transactions: 1000


,RecordDate,TransactionID,AccountID,TransactionDate,Type,Amount,Channel,Status,Type_std,Channel_std,Amount_num
0,2025-09-30,CITI-TXN-00001,CITI-ACC-0011,2025-09-28,purchase,1000,atm,NaN,Purchase,Atm,1000.0
1,2025-10-22,CITI-TXN-00002,CITI-ACC-0042,2025-09-28,Purchase,"AED 1,000",ATM,Pending,Purchase,Atm,1000.0
2,2025-07-12,CITI-TXN-00003,CITI-ACC-0054,2025-09-28,Purchase,NaN,NaN,NaN,Purchase,Nan,NaN


In [ ]:
# clean audit logs table
aud["RecordDate"] = to_date(aud["RecordDate"])
aud["ActionDate"] = to_date(aud["ActionDate"])

aud["AuditID"] = aud["AuditID"].astype(str).str.strip()

aud_clean = dedupe(aud, "AuditID")

print("✅ Clean Audit Logs:", len(aud_clean))
aud_clean.head(3)


✅ Clean Audit Logs: 500


,RecordDate,AuditID,CaseOrRequestID,ActionBy,ActionType,ActionDate,Remarks
0,2025-05-27,CITI-AUD-00001,CITI-SR-0150,CreditOfficer01,Rejected,2025-10-05 10:45:00,NaN
1,2025-06-25,CITI-AUD-00002,CITI-SR-0049,CSR_Alice,Escalated,2025-10-05 10:45:00,NaN
2,2025-10-02,CITI-AUD-00003,CITI-SR-0067,CSR_Alice,Rejected,2025-10-05 10:45:00,Followed up


In [ ]:
acct_clean = acct_clean[acct_clean["CustomerID"].isin(cust_clean["CustomerID"])]
cases_clean = cases_clean[cases_clean["AccountID"].isin(acct_clean["AccountID"])]
reqs_clean  = reqs_clean[reqs_clean["AccountID"].isin(acct_clean["AccountID"])]
txns_clean  = txns_clean[txns_clean["AccountID"].isin(acct_clean["AccountID"])]


In [ ]:

cust_clean.to_csv(f"{OUT_DIR}/dim_customer.csv", index=False)
acct_clean.to_csv(f"{OUT_DIR}/dim_account.csv", index=False)
cases_clean.to_csv(f"{OUT_DIR}/fact_cases.csv", index=False)
reqs_clean.to_csv(f"{OUT_DIR}/fact_service_requests.csv", index=False)
escal_clean.to_csv(f"{OUT_DIR}/fact_escalations.csv", index=False)
txns_clean.to_csv(f"{OUT_DIR}/fact_transactions.csv", index=False)
aud_clean.to_csv(f"{OUT_DIR}/fact_audits.csv", index=False)

print("✅✅ Clean CSVs created successfully!")


✅✅ Clean CSVs created successfully!


In [ ]:
if HAVE_SQLALCHEMY:
    engine = create_engine(f"sqlite:///{DB_PATH}")

    #  Dimension tables
    cust_clean.to_sql("dim_customer", engine, index=False, if_exists="replace")
    acct_clean.to_sql("dim_account", engine, index=False, if_exists="replace")

    #  Fact tables
    cases_clean.to_sql("fact_cases", engine, index=False, if_exists="replace")
    reqs_clean.to_sql("fact_service_requests", engine, index=False, if_exists="replace")
    escal_clean.to_sql("fact_escalations", engine, index=False, if_exists="replace")
    txns_clean.to_sql("fact_transactions", engine, index=False, if_exists="replace")
    aud_clean.to_sql("fact_audits", engine, index=False, if_exists="replace")

    print("🗄 SQLite Database created at:", DB_PATH)
else:
    print("⚠ SQLAlchemy not available → skipping DB export")


🗄 SQLite Database created at: /content/clean_outputs/citi_case_clean.db


In [ ]:
row_report = {
    "dim_customer": len(cust_clean),
    "dim_account": len(acct_clean),
    "fact_cases": len(cases_clean),
    "fact_service_requests": len(reqs_clean),
    "fact_escalations": len(escal_clean),
    "fact_transactions": len(txns_clean),
    "fact_audits": len(aud_clean)
}
print("📊 Final Row Counts →", row_report)


📊 Final Row Counts → {'dim_customer': 100, 'dim_account': 120, 'fact_cases': 150, 'fact_service_requests': 200, 'fact_escalations': 50, 'fact_transactions': 1000, 'fact_audits': 500}


In [ ]:

import os

for f in os.listdir("/content/clean_outputs"):
    print(f)


fact_service_requests.csv
citi_case_clean.db
dim_customer.csv
fact_escalations.csv
fact_audits.csv
fact_transactions.csv
fact_cases.csv
dim_account.csv


In [ ]:
from google.colab import files
files.download("/content/clean_outputs/fact_service_requests.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pd.read_csv("/content/clean_outputs/fact_transactions.csv").head()


,RecordDate,TransactionID,AccountID,TransactionDate,Type,Amount,Channel,Status,Type_std,Channel_std,Amount_num
0,2025-09-30,CITI-TXN-00001,CITI-ACC-0011,2025-09-28,purchase,1000,atm,NaN,Purchase,Atm,1000.0
1,2025-10-22,CITI-TXN-00002,CITI-ACC-0042,2025-09-28,Purchase,"AED 1,000",ATM,Pending,Purchase,Atm,1000.0
2,2025-07-12,CITI-TXN-00003,CITI-ACC-0054,2025-09-28,Purchase,NaN,NaN,NaN,Purchase,Nan,NaN
3,2025-07-09,CITI-TXN-00004,CITI-ACC-0088,2025-09-28,Purchase,"1,000.00",NaN,Failed,Purchase,Nan,1000.0
4,2025-08-10,CITI-TXN-00005,CITI-ACC-0004,2025-09-28,Payment,"AED 1,000",ATM,Successful,Payment,Atm,1000.0


In [2]:
!git remote -v
!git branch -a



fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
